# Notebook Overview

## Inputs
The input variables for the RNN in this notebook are the following:

    1. Time series data -- hours_l1, hours_l2, hours_l3, ..., hours_l30
    2. Additional predictors -- prov_id, day_of_week, week_perc0, week_perc6

## Model Architecture
The architecture used in this notebook combines an RNN with a feed forward neural network. The RNN layer recieves all 30 days of lagged shift data and makes a prediction for the shift on the 31st day. This prediction is then concatenated with the additional predictors and fed into a traditional neural network to generate a better prediction--the idea being that the RNN (through it's long and short term memory) learns patterns over time and the FF network adjusts these patterns based on additional information. 

In [1]:
import pandas as pd
import time
import tensorflow as tf

def normalize(dataset):
    seriesIndices = ['hours']
    for i in range(1,31):
        seriesIndices.append(f"hours_l{i}")
    dataset[seriesIndices] = dataset[seriesIndices].apply(lambda x: (x - x.mean()) / (x.std()))
    return dataset

include_fields = ['prov_id','hours','day_of_week','week_perc0','week_perc6']
for i in range(1,31):
    include_fields.append(f"hours_l{i}")

startTime = time.time()
train = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/training_set.csv",usecols=include_fields).dropna()
val = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/validation_set.csv",usecols=include_fields).dropna()
print(f"Loaded Train and Validation Sets. Time taken: {time.time()-startTime}")

Loaded Train and Validation Sets. Time taken: 421.74383902549744


In [2]:
#shuffle all data values so prov id distribution is same for train/val
import numpy as np
temp = pd.concat([train,val])
#Convert provider id's into dense representation
provider_map = {}
index = 0
for element in temp['prov_id'].unique():
    provider_map[element]=index
    index +=1
temp['prov_id'] = temp['prov_id'].map(provider_map)

np.random.shuffle(temp.values)
train = temp[:train.shape[0]]
val = temp[val.shape[0]:]
vocab_size = len(temp['prov_id'].unique())

In [3]:
train_inputs, train_labels = train.drop(['hours'], axis=1), train.filter(['hours'])
val_inputs, val_labels = val.drop(['hours'], axis=1), val.filter(['hours'])
#test_inputs, test_labels = test.drop(['hours'], axis=1), test.filter(['hours'])

In [4]:
def expand_one_hot(labels,dataset):
    outList = []
    for label in labels:  
        col = dataset[label]
        ###Generate a dict for all unique values (Don't waste space encoding non important job id's)
        map = {}
        index = 0
        for element in col.unique():
            map[element] = index
            index += 1
        col = col.map(map)
        tensor = tf.one_hot(col,len(col.unique()))
        outList.append(tensor)
        dataset = dataset.drop(columns=[label])
    
    outList.insert(0,dataset)
    output = tf.concat(outList,1)
    return output

train_inputs = expand_one_hot(['day_of_week'],train_inputs)
val_inputs = expand_one_hot(['day_of_week'],val_inputs)
#test_inputs = expand_one_hot(['day_of_week'],test_inputs)

print(train_inputs.shape)
print(train_inputs[0])

(124893352, 40)
tf.Tensor(
[ 0.    0.    7.51  0.    7.29  6.92  0.    0.    7.58  7.14  8.91  7.51
  0.    7.34  7.43  0.    7.33  8.58  7.82  7.62  0.    0.    7.51  8.46
  0.    8.14  7.66  7.55  8.51 10.33  8.41  0.12  0.11  1.    0.    0.
  0.    0.    0.    0.  ], shape=(40,), dtype=float32)


In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 256
trainSet = tf.data.Dataset.from_tensor_slices((train_inputs,train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valSet = tf.data.Dataset.from_tensor_slices((val_inputs,val_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
#testSet = tf.data.Dataset.from_tensor_slices((test_inputs,test_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(trainSet)

<BatchDataset shapes: ((None, 40), (None, 1)), types: (tf.float32, tf.float64)>


In [6]:
class RNN(tf.keras.Model):

    def __init__(self,vocab_size):
        super(RNN, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size,100)
        self.lstm = tf.keras.layers.LSTM(64)
        self.dense1 = tf.keras.layers.Dense(64, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(64, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(1)
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        #prov_id is first column
        embedding_vectors = self.embedding(inputs[:,0])
        time_series = tf.expand_dims(inputs[:,1:31],2)
        additional_inputs = inputs[:,31:]

        x = self.lstm(time_series)
        x = tf.concat([x,embedding_vectors,additional_inputs],1)
        x = self.dense1(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.dense2(x)
        if training:
            x = self.dropout(x, training=training)
       
        return self.out(x)

model = RNN(vocab_size)
    
model.compile(loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [7]:
import os
checkpoint_dir = './training_checkpointsRNN'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch+6}")

def decay(epoch):
  if epoch < 3:
    return 1e-4
  elif epoch >= 3 and epoch < 7:
    return 1e-5
  else:
    return 1e-6

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay)
]
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [7]:
model.fit(trainSet, epochs=10, callbacks=callbacks, validation_data=valSet)

NameError: name 'trainSet' is not defined